In [4]:
from utils.plotter import get_roc_curves_from_df
from utils.wandb_calls import fetch_files_from_runs
from utils import bad_runs
import os
import pandas as pd
import pickle
import shutil

### Download

In [23]:
for folder in os.listdir("./results/"):
    if "acdc" in folder or "sp" in folder:
        shutil.rmtree(f"./results/{folder}")


In [24]:
project = "circuit_discovery"
# group = "acdc"
files_to_fetch = ["result.pkl"]
fetch_files_from_runs(project, "", files_to_fetch)

100%|██████████| 735/735 [02:41<00:00,  4.56it/s]


### Process

In [2]:
from utils.get_circuit_discovery_scores import walk_dirs_and_get_scores

# walk_dirs_and_get_scores("acdc")
acdc_510_scores = walk_dirs_and_get_scores(weight=510, algorithm="acdc")
node_sp_510_scores = walk_dirs_and_get_scores(weight=510, algorithm="node_sp")

results/acdc_3/weight_510
results/acdc_4/weight_510
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
Skipping 4
results/acdc_11/weight_510
results/acdc_16/weight_510
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
results/acdc_21/weight_510
results/acdc_38/weight_510
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
Skipping 38
results/acdc_8/weight_510
results/acdc_24/weight_510
results/acdc_13/weight_510
results/node_sp_8/weight_510
results/node_sp_11/weight_510
results/node_sp_16/weight_510
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Skipping 16
Ski

/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/notebooks/utils/get_circuit_discovery_scores.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([
/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/notebooks/utils/get_circuit_discovery_scores.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([


In [6]:
node_sp_510_scores["run"].unique(), acdc_510_scores["run"].unique()

(array(['8', '11', '21', '3', '24', '13'], dtype=object),
 array(['3', '11', '21', '8', '24', '13'], dtype=object))

In [7]:
from sklearn import metrics

def make_fpr_tpr_monotonic(fpr_list, tpr_list):
    tpr_indices = [idx for idx in range(1, len(tpr_list)) if tpr_list[idx] > tpr_list[idx-1]]
    fpr_indices = [idx for idx in range(1, len(fpr_list)) if fpr_list[idx] > fpr_list[idx-1]]
    all_indices = list(set(tpr_indices + fpr_indices))
    # sort descending so we can remove from the end
    all_indices.sort(reverse=True)
    for idx in all_indices:
        del tpr_list[idx]
        del fpr_list[idx]
    return fpr_list, tpr_list

def get_ROCs(scores: pd.DataFrame, algorithm: str):
    if algorithm == "acdc":
        sort_key = "threshold"
        ascending = False
    elif "sp" in algorithm:
        sort_key = "lambda"
        ascending = True
    rocs = {} # run : {node: node_roc, edge: edge_roc}
    scores_by_run = scores.groupby("run")
    for run, run_scores in scores_by_run:
        run_scores = run_scores.sort_values(by=sort_key, ascending=ascending)
        # get 'node_tpr', 'node_fpr', 'edge_tpr', 'edge_fpr' from run_scores
        node_fpr = list(run_scores["node_fpr"].values.copy())
        node_tpr = list(run_scores["node_tpr"].values.copy())
        edge_fpr = list(run_scores["edge_fpr"].values.copy())
        edge_tpr = list(run_scores["edge_tpr"].values.copy())

        # make fpr and tpr lists monotonic
        node_fpr, node_tpr = make_fpr_tpr_monotonic(node_fpr, node_tpr)
        edge_fpr, edge_tpr = make_fpr_tpr_monotonic(edge_fpr, edge_tpr)

        
        # calculate roc_auc for node and edge
        node_roc = metrics.auc(node_fpr, node_tpr)
        edge_roc = metrics.auc(edge_fpr, edge_tpr)
        rocs[run] = {"node": node_roc, "edge": edge_roc}
    return rocs

In [8]:
get_ROCs(acdc_510_scores, "acdc"), get_ROCs(node_sp_510_scores, "node_sp")

({'11': {'node': 0.125, 'edge': 0.0},
  '13': {'node': 0.0, 'edge': 0.03260869565217391},
  '21': {'node': 0.05555555555555555, 'edge': 0.004901960784313725},
  '24': {'node': 0.05555555555555555, 'edge': 0.004901960784313725},
  '3': {'node': 0.875, 'edge': 0.0},
  '8': {'node': 0.3333333333333333, 'edge': 0.0}},
 {'11': {'node': 0.125, 'edge': 0.1568627450980392},
  '13': {'node': 0.5, 'edge': 0.4782608695652174},
  '21': {'node': 0.33333333333333337, 'edge': 0.47058823529411764},
  '24': {'node': 0.33333333333333337, 'edge': 0.47058823529411764},
  '3': {'node': 0.8571428571428572, 'edge': 0.8958333333333333},
  '8': {'node': 0.0, 'edge': 0.0}})

In [11]:
from utils.get_circuit_discovery_scores import append_row

acdc_rocs = get_ROCs(acdc_510_scores, "acdc")
node_sp_rocs = get_ROCs(node_sp_510_scores, "node_sp")
# make 2 combined dataframes: one for node_roc and one for edge_roc
node_roc_df = pd.DataFrame()
edge_roc_df = pd.DataFrame()

for k, v in acdc_rocs.items():
    run = k
    acdc_node_roc = v["node"]
    acdc_edge_roc = v["edge"]
    node_sp_node_roc = node_sp_rocs[k]["node"]
    node_sp_edge_roc = node_sp_rocs[k]["edge"]
    node_entry = {"run": run, "acdc": acdc_node_roc, "node_sp": node_sp_node_roc}
    edge_entry = {"run": run, "acdc": acdc_edge_roc, "node_sp": node_sp_edge_roc}
    node_roc_df = append_row(node_roc_df, pd.Series(node_entry))
    edge_roc_df = append_row(edge_roc_df, pd.Series(edge_entry))

(  run      acdc   node_sp
 0  11  0.125000  0.125000
 1  13  0.000000  0.500000
 2  21  0.055556  0.333333
 3  24  0.055556  0.333333
 4   3  0.875000  0.857143
 5   8  0.333333  0.000000,
   run      acdc   node_sp
 0  11  0.000000  0.156863
 1  13  0.032609  0.478261
 2  21  0.004902  0.470588
 3  24  0.004902  0.470588
 4   3  0.000000  0.895833
 5   8  0.000000  0.000000)

In [12]:
node_roc_df

,run,acdc,node_sp
0,11,0.125000,0.125000
1,13,0.000000,0.500000
2,21,0.055556,0.333333
3,24,0.055556,0.333333
4,3,0.875000,0.857143
5,8,0.333333,0.000000


In [13]:
edge_roc_df

,run,acdc,node_sp
0,11,0.000000,0.156863
1,13,0.032609,0.478261
2,21,0.004902,0.470588
3,24,0.004902,0.470588
4,3,0.000000,0.895833
5,8,0.000000,0.000000


### Realism

In [1]:
from utils.get_circuit_discovery_scores import get_realism_scores
acdc_510_realism = get_realism_scores(weight=510, algorithm="acdc")
node_sp_510_realism = get_realism_scores(weight=510, algorithm="node_sp")

/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/notebooks/utils/get_circuit_discovery_scores.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([
/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/notebooks/utils/get_circuit_discovery_scores.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([


In [5]:
def get_best_score(scores: pd.DataFrame):
    best_scores = {}
    scores_by_run = scores.groupby("run")
    for run, run_scores in scores_by_run:
        best_score = run_scores['score'].max()
        best_scores[run] = best_score
    return best_scores

acdc_510_best_realism = get_best_score(acdc_510_realism)
node_sp_510_best_realism = get_best_score(node_sp_510_realism)

In [9]:
combined_realism_df = pd.DataFrame()
for k, v in acdc_510_best_realism.items():
    run = k
    acdc_realism = v
    node_sp_realism = node_sp_510_best_realism[k]
    entry = {"run": run, "acdc": acdc_realism, "node_sp": node_sp_realism}
    combined_realism_df = append_row(combined_realism_df, pd.Series(entry))

combined_realism_df

,run,acdc,node_sp
0,11,0.474224,0.436851
1,13,0.674353,0.493942
2,21,1.000000,0.401581
3,24,1.000000,0.401581
4,3,0.357617,0.307617
5,8,0.269310,0.249977
